In [2]:
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

In [19]:
dataset = load_dataset("imagenet-1k")
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50")

inputs = image_processor(image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

In [14]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])